In [11]:
import pandas as pd
import numpy as np
import re
import csv
import sys

import time
import random
import distance
import random
import dynet as dy
import _dynet
import json
import time
from datetime import datetime

np.set_printoptions(threshold=np.nan)
dyparams = dy.DynetParams()
dyparams.set_mem(4096)

In [12]:
df = pd.read_csv('finale.txt', delimiter='\t', header=None)
df.columns = ['word1', 'word2', 'counts']

In [13]:
new_df = df[df.word1 != df.word2]

In [14]:
new_df.head()

,word1,word2,counts
0,20/400,betimol,125
1,fusidic acid,ondansetron,101
2,right ventricular hypertrophy,emotional distress,156
3,methylcellulose,seroquel,129
4,mannitol,patient date of birth,1046


In [15]:
new_df.loc[765526]

word1                                                pain
word2     pain adverse event by ctcae anatomic descriptor
counts                                            2120154
Name: 765526, dtype: object

In [16]:
def read_singleton_dict(dictionary_dir):
    dic, count, unigram = {}, 0, []
    with open(dictionary_dir, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for row in lines:
            line = row.split('\t')
            dic[line[0].strip()] = [int(line[1].strip()), count]
            unigram.append(count)
            count += 1
        return (dic, unigram)

In [17]:
path = '2_Singleton_Frequency_Counts/singlets_concepts_perBin_1d.txt'
singleton, unigram_table = read_singleton_dict(path)

In [18]:
def read_cofreq_dict(dictionary_dir):
    dic = {}
    with open(dictionary_dir, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for row in lines:
            line = row.split('\t')
            dic[(line[0].strip(), line[1].strip()) ] = int(line[2].strip())
        return dic

In [19]:
path = '1_Cofrequency_Counts/cofreqs_concepts_perBin_1d.txt'
cofreq = read_cofreq_dict(path)

In [20]:
sum(list(cofreq.values()))

13338466285

In [21]:
def build_training_data():
    training_data, count = [], 0
    
    for sent_a, sent_b in cofreq.keys():
        if sent_a != sent_b:
            indexA = singleton[sent_a][1]
            indexB = singleton[sent_b][1]
            count += 1
            training_data.append([indexA, indexB])
    return training_data

In [22]:
training_data = build_training_data()

In [23]:
# def is_valid_cuis(unigram, indexs, positive_cui):
#     for i in indexs:
#         if (unigram[i] == positive_cui):
#             return False
#     return True
def is_valid_cuis(unigram, indexs, positive_cui):
    for i in indexs:
        if (unigram[i] == positive_cui[0]) or (unigram[i] == positive_cui[1]) :
            return False
    return True

def get_negative_sample_cuis(unigram, sample_size, positive_cui):
    population = range(len(unigram))
    indexs = random.sample(population, sample_size)
    cuis = []
    while not is_valid_cuis(unigram, indexs, positive_cui):
        indexs = random.sample(population, sample_size)
    for i in indexs:
        cuis.append(unigram[i])
    return cuis

def insert_negative_sample(training_data, unigram, sample_size):
    for i in range(len(training_data)):
        # drop the old negative samples first
        word_list = training_data[i][:2]
        negative_samples = get_negative_sample_cuis(unigram, sample_size, word_list)
        training_data[i] = word_list + negative_samples

### Helper

In [24]:
def get_duration(start_time):
    end_time = datetime.now()
    return 'Duration: {}'.format(end_time - start_time)

### Dynet Network

In [25]:
# create a class encapsulating the network
class OurNetwork(object):
    def __init__(self, pc, dictionary):
        self.dict_size = len(dictionary)
        self.lookup_hidden = pc.add_lookup_parameters((self.dict_size, word_vector_size))
        self.lookup_output = pc.add_lookup_parameters((self.dict_size, word_vector_size))
        print('Create Network with word vector size: ' + str(word_vector_size))

    def __call__(self, inputs):
        # inputs: [input word, observed word, negative sample 1, negative sample 2...]
        input_word = inputs[0]
        observed_word = inputs[1]
        
        loss = dy.log(dy.logistic(self.get_dot_product(input_word, observed_word)))
        for negative_sample in inputs[2:]:
            loss += dy.log(dy.logistic(-self.get_dot_product(input_word, negative_sample)))
        return -loss
    
    def get_dot_product(self, wordA, wordB):
        return dy.dot_product(self.lookup_hidden[wordA], self.lookup_output[wordB])
    
    def create_network_return_loss(self, inputs):
        dy.renew_cg()
        return self(inputs)

In [26]:
word_vector_size = 300
m = dy.ParameterCollection()

# create network
network = OurNetwork(m, singleton)

Create Network with word vector size: 300


In [27]:
# create trainer
trainer = dy.SimpleSGDTrainer(m)

In [28]:
negative_sample_size = 3
print('Start training with negative sample size: ' + str(negative_sample_size))

total_loss = 0
global seen_instances, epoch
ts = int(time.time())
epoch = 1

Start training with negative sample size: 3


In [29]:
def run_iteration(it_num):
    seen_instances, total_loss = 0, 0
    start_time = datetime.now()
    print('Running Iteration: ' + str(it_num))
    
    np.random.shuffle(training_data)
    print('Finish shuffling data ' + get_duration(start_time))
    
    insert_negative_sample(training_data, unigram_table, negative_sample_size)
    print('Finish inserting negative samples' + get_duration(start_time))
    
    for word_list in training_data:
        seen_instances += 1
        loss = network.create_network_return_loss(word_list)
        total_loss += loss.value()
        loss.backward()
        trainer.update()
        if seen_instances > 1 and seen_instances % 10000 == 0:
            print("average loss is: " + str(total_loss / seen_instances) + ' '\
                  + get_duration(start_time) + ' ' + str(seen_instances))

In [31]:
seen_instances = 0
while True:
    run_iteration(epoch)
    epoch += 1
    if epoch == 11:
        save_embedding(network.lookup_hidden, ts)
    else:
        save_embedding(network.lookup_hidden, ts)

Running Iteration: 1
Finish shuffling data Duration: 0:00:02.011754
Finish inserting negative samplesDuration: 0:00:50.146772
average loss is: 2.773357973361015 Duration: 0:00:50.537210 10000
average loss is: 2.7738147804141047 Duration: 0:00:50.943733 20000
average loss is: 2.774008874797821 Duration: 0:00:51.329256 30000
average loss is: 2.774155233287811 Duration: 0:00:51.708247 40000
average loss is: 2.773873991189003 Duration: 0:00:52.106990 50000
average loss is: 2.773958459615707 Duration: 0:00:52.492557 60000
average loss is: 2.773896990445682 Duration: 0:00:52.883476 70000
average loss is: 2.773882171997428 Duration: 0:00:53.274395 80000
average loss is: 2.7738080310371185 Duration: 0:00:53.653478 90000
average loss is: 2.7738990778422354 Duration: 0:00:54.038963 100000
average loss is: 2.773723723990267 Duration: 0:00:54.437691 110000
average loss is: 2.7735324377616246 Duration: 0:00:54.828571 120000
average loss is: 2.773274233948267 Duration: 0:00:55.219529 130000
average 

average loss is: 1.8507557200197593 Duration: 0:01:37.459206 1210000
average loss is: 1.8463906257008766 Duration: 0:01:37.849595 1220000
average loss is: 1.842157879590549 Duration: 0:01:38.229202 1230000
average loss is: 1.8380761606181657 Duration: 0:01:38.612282 1240000
average loss is: 1.8338831920830683 Duration: 0:01:39.003210 1250000
average loss is: 1.8298040067632506 Duration: 0:01:39.403393 1260000
average loss is: 1.8257436493993773 Duration: 0:01:39.795976 1270000
average loss is: 1.8216690281848553 Duration: 0:01:40.185624 1280000
average loss is: 1.8176303046721936 Duration: 0:01:40.576041 1290000
average loss is: 1.8137353752018153 Duration: 0:01:40.976674 1300000
average loss is: 1.8098900827818643 Duration: 0:01:41.367116 1310000
average loss is: 1.8060083925361972 Duration: 0:01:41.747175 1320000
average loss is: 1.802246571379417 Duration: 0:01:42.145916 1330000
average loss is: 1.7986181868867614 Duration: 0:01:42.531378 1340000
average loss is: 1.7951256860480098 

average loss is: 1.554917406930707 Duration: 0:02:24.172378 2410000
average loss is: 1.553523875464898 Duration: 0:02:24.557876 2420000
average loss is: 1.552069525022875 Duration: 0:02:24.948795 2430000
average loss is: 1.5507123132479563 Duration: 0:02:25.339727 2440000
average loss is: 1.5494056520433839 Duration: 0:02:25.730617 2450000
average loss is: 1.54802595416538 Duration: 0:02:26.128548 2460000
average loss is: 1.5467759774896426 Duration: 0:02:26.521968 2470000
average loss is: 1.545342085287587 Duration: 0:02:26.917720 2480000
average loss is: 1.5440044565906672 Duration: 0:02:27.312403 2490000
average loss is: 1.542757212641511 Duration: 0:02:27.692496 2500000
average loss is: 1.541471873230525 Duration: 0:02:28.091224 2510000
average loss is: 1.540132554960076 Duration: 0:02:28.476816 2520000
average loss is: 1.5388619196956062 Duration: 0:02:28.883245 2530000
average loss is: 1.537576811113439 Duration: 0:02:29.266351 2540000
average loss is: 1.536369864735704 Duration:

average loss is: 1.4381543902427238 Duration: 0:03:11.214629 3610000
average loss is: 1.4374409973837232 Duration: 0:03:11.605545 3620000
average loss is: 1.4367137099120553 Duration: 0:03:11.984584 3630000
average loss is: 1.4360391652971751 Duration: 0:03:12.383304 3640000
average loss is: 1.4353514148015925 Duration: 0:03:12.766406 3650000
average loss is: 1.4346454919398735 Duration: 0:03:13.151887 3660000
average loss is: 1.4340606386337569 Duration: 0:03:13.549972 3670000
average loss is: 1.4333786254627665 Duration: 0:03:13.928648 3680000
average loss is: 1.432755478803594 Duration: 0:03:14.326952 3690000
average loss is: 1.4321422163943573 Duration: 0:03:14.717380 3700000
average loss is: 1.4315197549195322 Duration: 0:03:15.108296 3710000
average loss is: 1.4309014231346067 Duration: 0:03:15.493509 3720000
average loss is: 1.430256830968834 Duration: 0:03:15.884437 3730000
average loss is: 1.4295907878768437 Duration: 0:03:16.275308 3740000
average loss is: 1.428941245505137 D

average loss is: 1.3779811256808259 Duration: 0:03:58.237955 4810000
average loss is: 1.377616701157349 Duration: 0:03:58.628901 4820000
average loss is: 1.3772413380958073 Duration: 0:03:59.035037 4830000
average loss is: 1.3769120303152909 Duration: 0:03:59.438308 4840000
average loss is: 1.3765445174838018 Duration: 0:03:59.825241 4850000
average loss is: 1.3761434474977157 Duration: 0:04:00.226344 4860000
average loss is: 1.375798760838152 Duration: 0:04:00.619591 4870000
average loss is: 1.375414802007212 Duration: 0:04:01.005674 4880000
average loss is: 1.3749928064263968 Duration: 0:04:01.409818 4890000
average loss is: 1.3746318671081168 Duration: 0:04:01.814612 4900000
average loss is: 1.374231974011267 Duration: 0:04:02.205554 4910000
average loss is: 1.3738551090157063 Duration: 0:04:02.596507 4920000
average loss is: 1.3735188273091035 Duration: 0:04:02.987454 4930000
average loss is: 1.3731905965012319 Duration: 0:04:03.388170 4940000
average loss is: 1.3728229857403973 Du

...

average loss is: 1.2300646982310521 Duration: 0:03:39.059001 4230000
average loss is: 1.2300968630824431 Duration: 0:03:39.450937 4240000
average loss is: 1.230102630517066 Duration: 0:03:39.852144 4250000
average loss is: 1.2301582493051162 Duration: 0:03:40.231169 4260000
average loss is: 1.2301705522159225 Duration: 0:03:40.642194 4270000
average loss is: 1.2303468424480881 Duration: 0:03:41.035803 4280000
average loss is: 1.230424817886649 Duration: 0:03:41.440605 4290000
average loss is: 1.2304304691981414 Duration: 0:03:41.831531 4300000
average loss is: 1.2304412440289174 Duration: 0:03:42.232691 4310000
average loss is: 1.2305577825677982 Duration: 0:03:42.633910 4320000
average loss is: 1.2306527112985142 Duration: 0:03:43.024874 4330000
average loss is: 1.230696139652912 Duration: 0:03:43.430963 4340000
average loss is: 1.2306626995951413 Duration: 0:03:43.820613 4350000
average loss is: 1.2306973199341054 Duration: 0:03:44.221779 4360000
average loss is: 1.230710165853371 Du

average loss is: 1.2339394749923038 Duration: 0:04:26.395783 5430000
average loss is: 1.2340092830906009 Duration: 0:04:26.786414 5440000
average loss is: 1.2340942937152448 Duration: 0:04:27.192994 5450000
average loss is: 1.2341614672117667 Duration: 0:04:27.578606 5460000
average loss is: 1.2342179883908124 Duration: 0:04:27.984684 5470000
average loss is: 1.2343256243396437 Duration: 0:04:28.373944 5480000
average loss is: 1.2343311886119674 Duration: 0:04:28.775201 5490000
average loss is: 1.234358311182297 Duration: 0:04:29.165977 5500000
average loss is: 1.2343291844644813 Duration: 0:04:29.556802 5510000
average loss is: 1.234379827968868 Duration: 0:04:29.963402 5520000
average loss is: 1.2344120977045125 Duration: 0:04:30.348993 5530000
average loss is: 1.2344052045654594 Duration: 0:04:30.739933 5540000
average loss is: 1.2344065543287723 Duration: 0:04:31.140691 5550000
average loss is: 1.2344888956785816 Duration: 0:04:31.539445 5560000
average loss is: 1.234472420828483 D

average loss is: 1.2369700909759649 Duration: 0:05:13.842716 6630000
average loss is: 1.2369780370237642 Duration: 0:05:14.255784 6640000
average loss is: 1.236980224783615 Duration: 0:05:14.639650 6650000
average loss is: 1.2370472140024082 Duration: 0:05:15.046217 6660000
average loss is: 1.2370475008204072 Duration: 0:05:15.435423 6670000
average loss is: 1.2370698467910133 Duration: 0:05:15.827153 6680000
average loss is: 1.237065353620859 Duration: 0:05:16.216734 6690000
average loss is: 1.2370804477110058 Duration: 0:05:16.623267 6700000
average loss is: 1.2371327154011862 Duration: 0:05:17.014247 6710000
average loss is: 1.2371568290779547 Duration: 0:05:17.405112 6720000
average loss is: 1.2372262813409212 Duration: 0:05:17.796070 6730000
average loss is: 1.2372243981364626 Duration: 0:05:18.186845 6740000
average loss is: 1.237244748582117 Duration: 0:05:18.603355 6750000
average loss is: 1.237271994956512 Duration: 0:05:18.990965 6760000
average loss is: 1.2372908566704066 Du

average loss is: 1.239468825172237 Duration: 0:06:01.292708 7830000
average loss is: 1.239466796643656 Duration: 0:06:01.678309 7840000
average loss is: 1.2394863590735297 Duration: 0:06:02.081619 7850000
average loss is: 1.2394822383262298 Duration: 0:06:02.473392 7860000
average loss is: 1.2395308821329338 Duration: 0:06:02.858858 7870000
average loss is: 1.2395646502560376 Duration: 0:06:03.265258 7880000
average loss is: 1.2395604211217965 Duration: 0:06:03.650667 7890000
average loss is: 1.2395747226385914 Duration: 0:06:04.056772 7900000
average loss is: 1.2396074209827952 Duration: 0:06:04.443887 7910000
average loss is: 1.2396167990620448 Duration: 0:06:04.844923 7920000
average loss is: 1.2396160380692103 Duration: 0:06:05.234517 7930000
average loss is: 1.2396233257682536 Duration: 0:06:05.625340 7940000
average loss is: 1.2396289359603603 Duration: 0:06:06.016116 7950000
average loss is: 1.2396589921278638 Duration: 0:06:06.395062 7960000
average loss is: 1.2397081665491154 

KeyboardInterrupt: 

In [30]:
def get_embedding_dir(ts):
    base_dir = 'embeddings/'
    config_dir = base_dir + str(ts) + '_config' + '.txt'
    embedding_dir = base_dir + str(ts) + '_embedding' + '.txt'
    
    config = {'bin_category': 'singlets_concepts_perBin_1d.txt',
              'epoch': epoch,
              'training_data_size': len(training_data), 
              'negative_sample_size': negative_sample_size,
              'word_vector_size': word_vector_size,
              'embedding_dir': embedding_dir}
    
    with open(config_dir, 'w') as f:
        json.dump(config, f)
    return embedding_dir

def save_embedding(lookup, ts):
    embedding_dir = get_embedding_dir(ts)
    embedding_file = open(embedding_dir, "w")
    dictionary_keys = list(singleton.keys())
    line = str(len(dictionary_keys)) + ' ' + str(word_vector_size) + '\n'
    embedding_file.write(line)
    
    for i in dictionary_keys:
        cui, cui_index = i, singleton[i][1]
        weights = dy.lookup(lookup, cui_index).value()
        line = str(cui) + ' ' + ' '.join(map(str, weights)) + '\n'
        embedding_file.write(line)
    embedding_file.close()
    print('Embedding file saved at ' + embedding_dir)

Embedding file saved at embeddings/1524593616_embedding.txt
